In [10]:
import openai, os, json

In [11]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [12]:
def query(prompt, **kwargs):
    """
    wrapper for the API to save the prompt and the result
    """
    args = {
    "engine":"text-davinci-003",
    "temperature":0,
    "max_tokens":400,
    "stop":"END",
    }

    for key in kwargs:
        args[key] = kwargs[key]

    r = openai.Completion.create(prompt=prompt, **args)
    return r["choices"][0]["text"].strip()


In [5]:
class StoryWriter:

  def __init__(self, theme, genre, style):
    self.theme = theme
    self.genre = genre
    self.style = style

  def createStoryOverview(self):
    """
    Creates a story outline based on the theme, genre, and style
    """
    overviewForStoryPrompt = """ /* Generate an outline for a story

    type character{{
      "name": str,
      "profession": str,
      "age": int,
    }}

    interface outline {{
      "setting": str, //where the section takes place
      "characters": character[], //the character names
      "conflict": str, //what conflict needs to get solved
      "plotTwist": str, //any plot twists to occur
      "resolution": str // how the story concludes
    }}

    const generatePlot = (theme, genre, style) : outline =>{{
        return ai.generatePlot(theme)
    }}


    theme = '{}'
    genre = '{}'
    style = '{}'
    //make sure to include the keys in quotes so we can json.parse
    console.log("Out: " + generatePlot(theme, genre, style))
    Out:"""
    prompt = overviewForStoryPrompt.format(self.theme, self.genre, self.style)
    return query(prompt)



In [6]:
theme = "civil war"
genre = "movie"
style = "shakesepear"
createStoryOverview(theme, genre, style)

'{\n  "setting": "Civil War battlefield",\n  "characters": [\n    {\n      "name": "John",\n      "profession": "Soldier",\n      "age": 25\n    },\n    {\n      "name": "Jane",\n      "profession": "Nurse",\n      "age": 22\n    }\n  ],\n  "conflict": "John and Jane must find a way to survive the war and make it back home alive.",\n  "plotTwist": "John and Jane discover they are on opposite sides of the war.",\n  "resolution": "John and Jane find a way to make peace and survive the war together."\n}'

In [8]:
def createStoryOutline(storyOverview):
    """
    Creates a story outline based on the theme, genre, and style
    """
    prompt = outlineForStoryPrompt.format(storyOverview)
    return query(prompt)

outlineForStoryPrompt = """ /* Generate an outline for a story

type section{{
  "name": str, // the name of the section
  "description": str, // a brief overview of what happens in the section
}}

interface outline {{
  "parts": section[] //all of the sections in the story
}}

const generateOutline = (storyOverview) : outline =>{{
    return ai.generateOutline(storyOverview)
}}

overview = '{}'

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generateOutline(overview))
Out:"""

In [9]:
def fillCurrentSection(overview, outline, story, fnName, currentSection):
    fillOutlinePrompt = """ /* Write the next paragraph in a story for a given purpose

    //the overview of the entire story
    fullOverview = {}
    
    //the outline that the story should follow
    fullOutline = {}
    
    //the paragraphs of the story so far
    storySoFar = {}

    const {} = (instructions, currentOutlineSection) : str =>{{
        return ai.write()
    }}

    currentOutlineSection = {}

    console.log("Out: " + {}(currentOutlineSection))
    Out:""".format(overview, outline, story, fnName, currentSection, fnName)
    
    return fillOutlinePrompt

In [28]:
def generateNextStep (rawOverview, rawOutline, story):
    prompt = """//storyRouter.js Generate the name of the function and the inputs and outputs that it will take


story.rawOverview = {}

story.rawOutline = {}

story.current = {}



interface move{{
    "currentStatus": str, //the current status of the story, e.g. "building setting", "building characters", "writing story"
    "storyDirection": str, //the direction the story is going, e.g."building setting", "building down", "building to a climax"
    //the AI is capable of performing any task
    "nextFunction": str, //the name of the function to call, e.g. if building setting it would be "buildSetting", if writing story it would be "writeStory"
    "functionInputs": str, //the inputs the next function will need, e.g. {{character: "Romeo", emotion: "sad"}}; be creative
}}

const generateNextMove = () : move =>{{
    return ai.generateNextMove()
}}
    
console.log('Out: ' + generateNextMove())
//return 
Out:""".format(rawOverview, rawOutline, story)
    return prompt
    

In [39]:
def createPromptV2(overview, outline, story, fnName, currentSection, props):
    fillOutlinePrompt = """ /* Write the next paragraph in a story for a given purpose

    //the overview of the entire story
    fullOverview = {}
    
    //the outline that the story should follow
    fullOutline = {}
    
    //the paragraphs of the story so far
    storySoFar = {}

    const {} = (props) : str =>{{
        return ai.write()
    }}

    currentOutlineSection = {}

    console.log("Out: " + {}({}))
    Out:""".format(overview, outline, story, fnName, currentSection, fnName, props)
    
    return fillOutlinePrompt